kmp

Based on **Lauwens & Downey "Think Julia: How to Think Like a Computer Scientist"**
 
https://benlauwens.github.io/ThinkJulia.jl/latest/book.html**

Resources:

Julia webpage https://julialang.org/ 

Julia documentation https://docs.julialang.org/en/v1/


In [1]:
pwd()

"e:\\aaa-Julia-course-2023\\lectures-1.9"

## Chapter 13 -- Data structures -- Case studies

https://benlauwens.github.io/ThinkJulia.jl/latest/book.html#chap13

At this point you have learned about Julia’s core data structures, and you have seen some of the algorithms that use them. This chapter presents a case study with exercises that let you think about choosing data structures and practice using them.

### A usful detour

The alphabet -- ways of "stacking" ranges:

In [2]:
alpha1 = ['a' : 'z', 'A':'Z']

2-element Vector{StepRange{Char, Int64}}:
 'a':1:'z'
 'A':1:'Z'

In [3]:
alphabet = ['a' : 'z'; 'A':'Z']

52-element Vector{Char}:
 'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)
 'b': ASCII/Unicode U+0062 (category Ll: Letter, lowercase)
 'c': ASCII/Unicode U+0063 (category Ll: Letter, lowercase)
 'd': ASCII/Unicode U+0064 (category Ll: Letter, lowercase)
 'e': ASCII/Unicode U+0065 (category Ll: Letter, lowercase)
 'f': ASCII/Unicode U+0066 (category Ll: Letter, lowercase)
 'g': ASCII/Unicode U+0067 (category Ll: Letter, lowercase)
 'h': ASCII/Unicode U+0068 (category Ll: Letter, lowercase)
 'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)
 'j': ASCII/Unicode U+006A (category Ll: Letter, lowercase)
 ⋮
 'R': ASCII/Unicode U+0052 (category Lu: Letter, uppercase)
 'S': ASCII/Unicode U+0053 (category Lu: Letter, uppercase)
 'T': ASCII/Unicode U+0054 (category Lu: Letter, uppercase)
 'U': ASCII/Unicode U+0055 (category Lu: Letter, uppercase)
 'V': ASCII/Unicode U+0056 (category Lu: Letter, uppercase)
 'W': ASCII/Unicode U+0057 (category Lu: Letter, uppercase)
 'X': ASCII/

In [4]:
alpha2 = ['a' : 'z' 'A':'Z']

26×2 Matrix{Char}:
 'a'  'A'
 'b'  'B'
 'c'  'C'
 'd'  'D'
 'e'  'E'
 'f'  'F'
 'g'  'G'
 'h'  'H'
 'i'  'I'
 'j'  'J'
 ⋮    
 'r'  'R'
 's'  'S'
 't'  'T'
 'u'  'U'
 'v'  'V'
 'w'  'W'
 'x'  'X'
 'y'  'Y'
 'z'  'Z'

In [5]:
chr_index = Dict(zip(alphabet, Int.(alphabet)))	

Dict{Char, Int64} with 52 entries:
  'n' => 110
  'f' => 102
  'w' => 119
  'E' => 69
  'Z' => 90
  'o' => 111
  'B' => 66
  'C' => 67
  'h' => 104
  'i' => 105
  'r' => 114
  't' => 116
  'q' => 113
  'M' => 77
  'K' => 75
  'J' => 74
  'P' => 80
  'I' => 73
  'H' => 72
  ⋮   => ⋮

In [6]:
letter_index = Dict(zip(string.(alphabet), Int.(alphabet)))

Dict{String, Int64} with 52 entries:
  "F" => 70
  "f" => 102
  "Z" => 90
  "c" => 99
  "e" => 101
  "C" => 67
  "P" => 80
  "T" => 84
  "x" => 120
  "W" => 87
  "r" => 114
  "L" => 76
  "O" => 79
  "B" => 66
  "M" => 77
  "d" => 100
  "i" => 105
  "y" => 121
  "g" => 103
  ⋮   => ⋮

The function **isletter** tests whether a character is alphabetic.

In [7]:
# built in version

isletter('B')

true

In [9]:
function is_letter(chr::String)	
    alphabet = string.(['a' : 'z'; 'A':'Z'; '-'; ' '])
    return chr in alphabet
end

is_letter("B")

true

Let us download "Emma" by Jane Austen (https://ia800303.us.archive.org/24/items/EmmaJaneAusten_753/) from the Internet Archive (https://ia800303.us.archive.org/) in raw form and see what it contains.

In [10]:
# raw read of Emma

em2 = let
	url = "https://ia800303.us.archive.org/24/items/EmmaJaneAusten_753/emma_pdf_djvu.txt"
	raw_text = read(download(url), String);
	lowercase.(raw_text)
end

"* + '^^p* \n\n\n\ntemnt \n\n\n\nillustrhted \nby \n\nhugh thomson \n\n\n\n#: \n\n\n\nyj \n\n\n\na \n\n\n\n£ \n\n\n\n■4 \n\n\n\n£ \n\n\n\ndigitized by the internet archive \n\nin 2008 with funding from \n\nmicrosoft corporation \n\n\n\nhttp://www.archive.org/details/emmaaustenooaustrich \n\n\n\n\nemma ivas not sorry to h" ⋯ 938448 bytes ⋯ "dictions of the small band of true friends who witnessed the \nceremony, were fully answered in the perfect happiness of the \nunion. \n\n\n\nthe end \n\n\n\nprinted by r. & r. clark, limited, edinburgh \n\n\n\n\nrs \n\n\n\n\n\n\n\n\nc) \n\n\n\nkkvt^v: \n\n\n\ns§ \n\n\n\n\n\n\n\ni \n\n\n\nth \n\n\n\n\n^^^m \n\n\n\n\n\n\n\n\n\n"

### Exercise 13-1

Write a program that reads a file, breaks each line into words, strips whitespace and punctuation from the words, and converts them to lowercase. Clearly we are going to need to do some pre-processing of the raw data ("data wrangling").

In [11]:
function preprocess(text::String)
    # see info on regular expressions
    # https://docs.julialang.org/en/v1/manual/strings/#Regular-Expressions
    # http://www.pcre.org/current/doc/html/pcre2syntax.html

    # clean up whitespace
    text = replace(text, r"\s+" => " ")

    # clean up broken words: "com-", "fortable"
    text = replace(text, r"-\s+" => "")

    # clean up other characters found by random inspection
    text = replace(text, r"(\.|,|;|:|\?|!|'|\-|—|\^|)" => "")
    text = replace(text, r"(\[|\]|\)|\(|\{|\}|•|\*|\<|\>)" => "")
    text = replace(text, r"(°|■|©|»|«|♦|•|\"|£|\$|&|\/)" => "")
    text = replace(text, r"(\+|\#|§)" => "")
    text = replace(text, r"([0-9])" => "")

    #generate a CSV text 
    cleantext = replace(text, r"(\s|\b)" => ",")

    return cleantext 	# sort of ...
end

function splt(txt::String)
    # split on whitespace or other word boundaries
    # and type-cast the Array of Substrings from split into an Array of Strings
    # tokens = string.(split(cleantext, r"(\s|\b)"))

    return tokens = string.(split(txt, ","))
end


#---

words_em2 = splt(preprocess(em2))  # still not good enough


172958-element Vector{String}:
 ""
 ""
 "p"
 "temnt"
 "illustrhted"
 "by"
 "hugh"
 "thomson"
 ""
 "yj"
 ⋮
 "edinburgh"
 "rs"
 "c"
 "kkvtv"
 "s"
 "i"
 "th"
 "m"
 ""

### Word frequency analysis

### Exercise 13-2

Let's continue with Emma, but if you prefer, you could go to Project Gutenberg (https://gutenberg.org) and download your favorite out-of-copyright book in plain text format. 

Here is a slightly different approach to the initial `data wrangling` step which skips over the header information and other irrelevant information at the beginning and the end of the file, and process the rest of the words as before.

Note the management of unwanted features like com- \nfortable etc.

In [12]:
emma = let
	
	url = "https://ia800303.us.archive.org/24/items/EmmaJaneAusten_753/emma_pdf_djvu.txt"

	raw_text = read(download(url), String);
	
	first_words = "Emma Woodhouse";
	last_words = "THE END";
	
	start_index = findfirst(first_words, raw_text)[1]
	stop_index = findlast(last_words, raw_text)[end]
	
	lowercase.(raw_text[start_index:stop_index])
	
end

"emma woodhouse, handsome, clever, and rich, with a com- \nfortable home and happy disposition, seemed to unite some \nof the best blessings of existence ; and had lived nearly \ntwenty-one years in the world with very little to distress or \nvex her. \n\nshe was the youngest" ⋯ 917235 bytes ⋯ "elina would stare when she heard of it.' but, in spite of \nthese deficiencies, the wishes, the hopes, the confidence, the \npredictions of the small band of true friends who witnessed the \nceremony, were fully answered in the perfect happiness of the \nunion. \n\n\n\nthe end"

In [13]:
function splitwords(text::String)
	# see info on regular expressions
	# https://docs.julialang.org/en/v1/manual/strings/#Regular-Expressions
	# http://www.pcre.org/current/doc/html/pcre2syntax.html
	
	# clean up whitespaces
	text = replace(text, r"\s+" => " ")
	
	# clean up broken words: "com-", "fortable"
	text = replace(text, r"-\s+" => "")
	text = replace(text, r"(\.|,|;|:|\?|!|'|\-|—|\^|§)" => "")
	text = replace(text, r"(\[|\]|\)|\(|\{|\}|•|\*|\<|\>)" => "")
	text = replace(text, r"(°|■|©|»|«|♦|•|\"|£|\$|&|\/)" => "")
	text = replace(text, r"x*" => "")
	text = replace(text, r"([0-9])" => "")
	
	#text = replace(text, r"(\s|\b)" => ",")
	cleantext = replace(text, r"(\s|\b)" => ",")
	#cleantext = replace(text, r"\\" => "")

	# split on whitespace or other word boundaries
	# and type-cast the Array of Substrings from split into an Array of Strings
	#tokens = String.(split(cleantext, r"(\s|\b)"))
	tokens = string.(split(cleantext, ","))
end

emma_words = splitwords(emma)

169369-element Vector{String}:
 ""
 "emma"
 "woodhouse"
 "handsome"
 "clever"
 "and"
 "rich"
 "with"
 "a"
 "comfortable"
 ⋮
 "the"
 "perfect"
 "happiness"
 "of"
 "the"
 "union"
 "the"
 "end"
 ""

Modify the program to count the total number of words in the book, and the number of times each word is used. Print the number of different words used in the book. You could compare different books by different authors, written in different eras. Who of the authors you looked at uses the most extensive vocabulary?

In [14]:
function word_counts(words::Array{String, 1})
	
	counts = Dict{String,Int}()

	for word in words
		
		if word == "" || word == "\\"
			# do nothing
		else
			counts[word] = get!(counts, word, 0) + 1
		end
	end
	
	return counts
end

emma_dictionary = word_counts(emma_words)

Dict{String, Int64} with 7413 entries:
  "prejudices"      => 1
  "practise"        => 3
  "adviser"         => 1
  "forbade"         => 1
  "offend"          => 3
  "quicksighted"    => 1
  "contemplate"     => 1
  "enjoy"           => 9
  "diffuse"         => 1
  "unreserved"      => 1
  "transplantation" => 1
  "fight"           => 1
  "dulness"         => 3
  "everywhere"      => 3
  "indelicacy"      => 1
  "inattentive"     => 1
  "wellbred"        => 2
  "helping"         => 1
  "whose"           => 39
  ⋮                 => ⋮

In [15]:
function reverse_dictionary(d::Dict)
	D = Dict{Int,Array{String,1}}()
	
	for key in keys(d)
		
		D[d[key]] = push!( get!(D, d[key], []) , key)
	end
	
	return D
end

reverse_emma_dictionary = reverse_dictionary(emma_dictionary)

Dict{Int64, Vector{String}} with 262 entries:
  56   => ["name", "yourself", "days", "walk", "leave", "isabella", "state"]
  35   => ["got", "marriage", "please", "child", "taste", "company"]
  1033 => ["at"]
  60   => ["thinking", "through", "talk", "others", "children", "westons", "bad…
  220  => ["other", "shall"]
  734  => ["my"]
  67   => ["ready", "three", "pretty", "perfectly", "lady"]
  73   => ["whole", "whom", "able", "between"]
  251  => ["has", "about"]
  115  => ["still"]
  112  => ["sort", "something", "moment"]
  1437 => ["as"]
  86   => ["both", "room"]
  168  => ["anything"]
  364  => ["when"]
  207  => ["out"]
  242  => ["dear"]
  551  => ["there"]
  12   => ["entreat", "beg", "drive", "rate", "proceed", "awkward", "seated", "…
  ⋮    => ⋮

### Exercise 13-3

Modify the program from the previous exercise to print the 10 most frequently used words in the book.

In [16]:
function sort_on_vs(d::Dict, rev=true)
    ks = collect(keys(d))
    vs = collect(values(d))

    if rev 
        sindex = sortperm(vs, rev=true) 	
    else
        sindex = sortperm(vs, rev=false)
    end

    sorted_ks = ks[sindex]	# keys, values returned in the same order
    sorted_vs = vs[sindex]

    return collect(zip(sorted_ks, sorted_vs))
end

#---

N = 10
		
list = sort_on_vs(emma_dictionary)[1:N]

println("List of the $N most common words in ", "Emma")
println("----------------------------------------")

for k = 1:length(list)
    println("$k.", "\t ", list[k][1], "   \t\t", list[k][2])
end

List of the 10 most common words in Emma
----------------------------------------
1.	 the   		5209
2.	 to   		5195
3.	 and   		4889
4.	 of   		4292
5.	 i   		3196
6.	 a   		3131
7.	 it   		2530
8.	 her   		2487
9.	 was   		2406
10.	 she   		2365


### Exercise 13-4

Modify the previous program to read a word list and then print all the words in the book that are not in the word list. How many of them are typos? How many of them are common words that should be in the word list, and how many of them are really obscure?


In [17]:
wordlist = let  # note that the output of a let-block is assigned to a variable
	
	words = Dict{String, Any}()
	# note the escape character \ in \\
	fullpath = "E:\\Julia\\a-Pluto-NoteBooks\\ThinkJulia-notebooks\\words.txt"
	infile = open(fullpath)

	for line in eachline(infile)
		words[line] = []
	end
	
	close(infile)
	words
	
end

Dict{String, Any} with 113809 entries:
  "epinaoi"         => Any[]
  "nimbused"        => Any[]
  "pintoes"         => Any[]
  "interties"       => Any[]
  "inattentive"     => Any[]
  "cliquing"        => Any[]
  "photosynthesis"  => Any[]
  "sleepwalking"    => Any[]
  "chicanes"        => Any[]
  "lunk"            => Any[]
  "ethmoids"        => Any[]
  "reemitted"       => Any[]
  "henry"           => Any[]
  "hotheadednesses" => Any[]
  "planches"        => Any[]
  "entomb"          => Any[]
  "whiz"            => Any[]
  "redresses"       => Any[]
  "wormwoods"       => Any[]
  ⋮                 => ⋮

In [18]:
function dictionary_difference(book::Dict, list::Dict)
	
    (ismissing(book) || ismissing(wordlist)) && return missing  # guard

    words_not_in_list = Array{String, 1}()

    for word in keys(book)

        if word in keys(list)
            # do nothing
        else
            push!(words_not_in_list, word)
        end		
    end		
    return words_not_in_list
end

#---

nonwords = dictionary_difference(emma_dictionary, wordlist)
sort!(nonwords)

for nonword in nonwords
    if length(nonword) < 2
        println()
    end
    println(nonword)
end


a
abbeymill
abbotts
abdys


abruptness
acquirements
acquittal
adair
adelaide
administered
admirable
admirably
adventuring
adversarys
affi
affied
aggrandise
agreeably
aint
al
aladdins
alderneys
allsufficiency
amiableness
amor
andfeel
aniety
anious
aniously
anne
anothers
anybodys
apartment
apartments
apologise
apologised
appellation
appledumplings
appletart
appletrees
april
arevery
arrowroot
arthur
asparagus
astleys
augusta
authorised
avoodhouse
aweek
azvayfrom

b
backgammontable
balycraig
barnes
barouchelandau
bateses
batess
behaviour
behindhand
bella
bellas
bestjudging
besttempered
bickerton
birmingham
blockhead
boardingschool
bowwindow
bragge
bragges
braithwaites
bridepeople
broadway
broadwoods
brotherinlaw
brotherinlaws
brunswick
buyings

c
campbell
campbells
campfever
cara
cardparties
cardroom
cardtable
caro
caroline
carpetwork
carriagehorses
carteblanche
catherine
ceaseless
ceiting
charitable
checkerwork
cheerfultempered
childs
christian
christmas
churchhill
churchill
churchills
churchilps
churchwardens
clar

himc
hindrance
hodges
holyhead
homebaked
homefarm
honourable
honourably
hoodwinkd
horrorstruck
housebreaking
houseroom
hppe
hughes
humourist
hyperbolical

i
ii
iii
ike
illassorted
illbred
illdisposed
illequipped
illhealth
illhumour
illiberal
illjudged
illluck
illtempered
illtimed
illwill
ilr
imaginarydeclaration
imaginist
impartial
improvidently
inbetweens
incommoded
incommoding
inconsideration
incumbrance
indiscreetly
inebriety
inecusable
ineperience
inepressible
inepressibly
inferiorities
influenced
injustice
inquietudes
inseparably
insignificance
introd
io
ioo
ioooo
ireland
irish
irksomeness
irresistibly
isabella
isabellas
italian
iter
iv
izm

j
james
jamess
january
jeffereys
jiad
jiear
jlies
jrompt
jt
july
jum
june

k
kinghtley
kingston
kingsweston
knighterrantry
knightley
knightleys

l
ladys
langham
largesized
larkiis
larkins
larkinss
latters
leastin
leavetaking
letterboy
lias
lii
liii
linendraper
liorseback
liv
london
longestsighted
longstanding
luuries
luurious
luury
lv
lvi
lvii


unepectedly
unepensively
unequalled
unfastidious
unfavourable
unfeignedly
unfrequently
ungallant
ungracious
ungraciously
unimpeded
uninterruptedly
unmentioned
unmied
unmirthful
unmodulated
unobjectionable
unostentatious
unpardonably
unperceived
unpermitted
unpersuadable
unpolished
unpretending
unprovided
unreserve
unreserved
unseasonableness
unsentimental
unsuccessfully
unsuitable
unsullied
unsuspected
unsuspicious
untainted
untouched
untowardly
urahy
usj

v
valetudinarian
vanitybaits
ve
veation
veatious
veed
venice
vi
vii
viii
visitings

w
wacandles
wakefield
wallis
wallises
warmhearted
watercolours
wateringplace
waverings
weddingcake
weddingday
weddingvisit
wellbehaved
wellbred
wellbuilt
welldoing
welleducated
wellgrown
wellinformed
welljudging
wellknown
welllooking
wellmeaning
welltied
wellwritten
weredisturbed
wesioii
weston
westons
weymouth
whereever
whistclub
whistplayers
wholelength
wholelengths
widowerfather
william
williams
wiltshire
windowcurtain
windsor
wingfield
wingfields

### Random numbers

Given the same inputs, most computer programs generate the same outputs every time, so they are said to be deterministic. For some applications, though, we want the computer to be unpredictable. Making a program nondeterministic turns out to be difficult, but there are ways to make it at least behave as if nondeterministic. One of them is to use algorithms that generate **pseudorandom numbers**. 

Pseudorandom numbers are not truly random because they are generated by a deterministic computation, but just by looking at the numbers it is all but impossible to distinguish them from random. 

The function **rand** returns a random float64 between 0.0 and 1.0 (including 0.0 but not 1.0). Each time you call `rand`, you get the next number in a long series. To see a sample, run this loop:

```Julia
	for i in 1:10
		x = rand()
		println(x)
	end
```

In [34]:
println("Pseudorandom numbers generated by rand()")
println("----------------------------------------")
for i in 1:10
    x = rand()
    println(i, ".\t\t", x)
end

Pseudorandom numbers generated by rand()
----------------------------------------


1.		0.9660379093226862
2.		0.3447139260291344
3.		0.32362659057940213
4.		0.11518839518739099
5.		0.44468825318179717
6.		0.018807458463036775
7.		0.9258356277707761
8.		0.4946537572509965
9.		0.9340204015303589
10.		0.1848867268243446


The function **rand** can take an **iterator or array as argument** and returns a random element:

```Julia
	for i in 1:10
		x = rand(1:6)
		print(x, " ")
	end
```

In [20]:
fruits = ['🍌', '🍎', '🍐', '🍓', '🍒', '🍋'] 
	
println("Pseudorandom numbers generated by rand()")
println("----------------------------------------")
println()
for i in 1:10
    x = rand(1:6)
    y = rand('a':'f')
    z = rand(fruits)
    println(i, ".\t", x, "\t", y, "\t", z)
end

Pseudorandom numbers generated by rand()
----------------------------------------



1.	4	b	🍎
2.	6	a	🍎
3.	1	d	🍐
4.	5	c	🍎
5.	3	b	🍒
6.	4	e	🍋
7.	6	a	🍓
8.	1	f	🍐
9.	4	d	🍎
10.	4	d	🍎


### Exercise 13-5

Write a function named `choosefromhist` that takes a histogram as defined in `Histogram - a Collection of Counters` and returns a random value from the histogram, chosen with probability in proportion to frequency.

For example, for this histogram:

```Julia
	julia> t = ['a', 'a', 'b'];

	julia> histogram(t)
		Dict{Any,Any} with 2 entries:
		  'a' => 2
		  'b' => 1
```

Your function should return 'a' with probability 2/3 and 'b' with probability 1/3.


### Word Histogram

You should attempt the previous exercises before you go on. You will also need https://github.com/BenLauwens/ThinkJulia.jl/blob/master/data/emma.txt. Here is a program that reads a file and builds a histogram of the words in the file:

```Julia
	function processfile(filename)
		hist = Dict()
		for line in eachline(filename)
			processline(line, hist)
		end
		hist
	end;

	function processline(line, hist)
		line = replace(line, '-' => ' ')
		for word in split(line)
			word = string(filter(isletter, [word...])...)
			word = lowercase(word)
			hist[word] = get!(hist, word, 0) + 1
		end
	end;

	hist = processfile("emma.txt")
```

In [21]:
function processfile(filename)
	hist = Dict()

	for line in eachline(filename)
		processline(line, hist)
	end
	return hist
end

processfile (generic function with 1 method)

In [22]:
function processline(line, hist)
	line = replace(line, '-' => ' ')

	for word in Base.split(line)		
		word = string(filter(isletter, [word...])...)
		word = lowercase(word)
		hist[word] = get!(hist, word, 0) + 1
	end
end

processline (generic function with 1 method)

In [23]:
url = "https://github.com/BenLauwens/ThinkJulia.jl/blob/master/data/emma.txt"
	
filename = open(download(url))

em3 = processfile(filename)

close(filename)

em3

Dict{Any, Any} with 18241 entries:
  "heardrby"              => 1
  "reanimationrof"        => 1
  "hartfieldri"           => 1
  "indelicacy"            => 1
  "inattentive"           => 1
  "gout"                  => 1
  "henry"                 => 16
  "devotedrto"            => 1
  "promiseri"             => 1
  "particularlyrsilent"   => 1
  "couldrmanage"          => 1
  "incommoded"            => 1
  "stronglyrrecommending" => 1
  "bateses"               => 4
  "pianoforte"            => 16
  "rises"                 => 1
  "unwholesome"           => 2
  "argirl"                => 1
  "berenchanted"          => 1
  ⋮                       => ⋮

This program reads emma.txt, which contains the text of Emma by Jane Austen.

`processfile` loops through the lines of the file, passing them one at a time to processline. The histogram hist is being used as an accumulator. `processline` uses the function replace to replace hyphens with spaces before using split to break the line into an array of strings. It traverses the array of words and uses `filter`, `isletter` and `lowercase` to remove punctuation and convert to lower case. (It is a shorthand to say that strings are “converted”; remember that strings are immutable, so a function like lowercase return new strings.) Finally, `processline` updates the histogram by creating a new item or incrementing an existing one. 

To count the total number of words in the file, we can add up the frequencies in the histogram:

```Julia
	function totalwords(hist)
		sum(values(hist))
	end
```

The number of different words is just the number of items in the dictionary:

```Julia
	function differentwords(hist)
		length(hist)
	end
```

Here is some code to print the results:

```Julia
	julia> println("Total number of words: ", totalwords(hist))
		Total number of words: 162742

	julia> println("Number of different words: ", differentwords(hist))
		Number of different words: 7380
```

### Most common words

To find the most common words, we can make an array of tuples, where each tuple contains a word and its frequency, and sort it. 

The following function takes a histogram and returns an array of word-frequency tuples:

```Julia
	function mostcommon(hist)
		t = []
		for (key, value) in hist
			push!(t, (value, key))
		end
		reverse(sort(t))
	end
```

In each tuple, the frequency appears first, so the resulting array is sorted by frequency. 

Here is a loop that prints the 10 most common words:

```Julia
	t = mostcommon(hist)
	println("The most common words are:")
	for (freq, word) in t[1:10]
		println(word, "\t", freq)
	end
```

The tab character ('\t') is used as a “separator”, rather than a space, so the second column is lined up. Here are the results from Emma; the most common words are:

```Julia
	to	5295
	the	5266
	and	4931
	of	4339
	i	3191
	a	3155
	it	2546
	her	2483
	was	2400
	she	2364
```
Let's check how this works!

In [24]:
function mostcommon(hist)
    t = []
    for (key, value) in hist
        push!(t, (value, key))
    end
    reverse(sort(t))
end
    
t = mostcommon(em3)

#==
println("The most common words in EM3:")
println("----------------------------")
k = 0
for (freq, word) in t[1:10]
    k += 1
    println("$k.\t", word, "\t\t\t", freq)
end
==#

18241-element Vector{Any}:
 (4675, "to")
 (4639, "the")
 (3862, "and")
 (3767, "of")
 (2840, "a")
 (2492, "i")
 (2148, "was")
 (2124, "her")
 (2032, "it")
 (1911, "not")
 ⋮
 (1, "ablerof")
 (1, "abide")
 (1, "abhorred")
 (1, "abdys")
 (1, "abbots")
 (1, "abbeyrrthank")
 (1, "abbeyrrrrchapter")
 (1, "abbeyrbut")
 (1, "abbeyrand")

In [25]:
function clipped_t(hist)
	
    k = 0

    for j = 1:length(hist)
    
        if hist[j][1] > 6000
            k += 1
        end			
    end
return hist[k+1:end]
end


t = mostcommon(em3)
		
t = clipped_t(t)

println("The most common words in clipped EM3:")
println("-------------------------------------")
k = 0
for (freq, word) in t[1:11]
    if word == ""
        # do nothing
    else
        k += 1
        println("$k.\t\t", word, "    \t\t", freq)
    end
end

The most common words in clipped EM3:
-------------------------------------


1.		to    		4675
2.		the    		4639
3.		and    		3862
4.		of    		3767
5.		a    		2840
6.		i    		2492
7.		was    		2148
8.		her    		2124
9.		it    		2032
10.		not    		1911
11.		in    		1903


From the book:

```Julia
		to	5295
		the	5266
		and	4931
		of	4339
		i	3191
		a	3155
		it	2546
		her	2483
		was	2400
		she	2364
```

Why do the results differ?

In [26]:
N = 10
		
list = sort_on_vs(word_counts(words_em2))[1:N]

println("List of the $N most common words in ", "EM2")
println("---------------------------------------")

for k = 1:length(list)
    println("$k.", "\t ", list[k][1], "   \t\t", list[k][2])
end

List of the 10 most common words in EM2
---------------------------------------
1.	 the   		5433
2.	 to   		5285
3.	 and   		4982
4.	 of   		4422
5.	 i   		3198
6.	 a   		3182
7.	 it   		2559
8.	 her   		2532
9.	 was   		2434
10.	 she   		2395


### Optional parameters

We have seen built-in functions that take optional arguments. It is possible to write programmer-defined functions with **optional arguments** too. 

Here is a function that prints the most common words in a histogram:

```Julia
	function printmostcommon(hist, num=10)
		t = mostcommon(hist)
		println("The most common words are: ")
		for (freq, word) in t[1:num]
			println(word, "\t", freq)
		end
	end
```

**The first parameter is required; the second is optional.** The **default value** of `num` is 10. If you only provide one argument:

```Julia
	printmostcommon(hist)
```

`num` gets the default value. 

If you provide two arguments:

```Julia
	printmostcommon(hist, 20)
```

`num` gets the value of the argument instead. 

The optional argument overrides the default value. 

If a function has both required and optional parameters, all the required parameters have to come first, followed by the optional ones.

### Dictionary subtraction

Finding the words from the book that are not in the word list from `words.txt` is a problem you might recognize as **set subtraction**; that is, we want to find all the words from one set (the words in the book) that are not in the other (the words in the list).

`subtract` takes dictionaries d1 and d2 and returns a new dictionary that contains all the keys from d1 that are not in d2. Since we do not really care about the values, we set them all to nothing:

In [27]:
function subtract(d1::Dict, d2::Dict)
	result = Dict()
	
	for key in keys(d1)
		if key ∉ keys(d2)
			result[key] = nothing
		end
	end
	
	return collect(keys(result))
end

subtract (generic function with 1 method)

In [28]:
subtract(emma_dictionary, wordlist)

920-element Vector{Any}:
 "epediency"
 "quicksighted"
 "outwardly"
 "adelaide"
 "unreserved"
 "eerting"
 "wellbred"
 "mma"
 "andfeel"
 "incommoded"
 ⋮
 "unfastidious"
 "finestlooking"
 "criticising"
 "ecesses"
 "eactly"
 "coes"
 "supperroom"
 "eisted"
 "fairfas"

To find the words in the book that are not in `words.txt`, we can use processfile to build a histogram for `words.txt`, and then subtract:

```Julia
	words = processfile("words.txt")
	diff = subtract(hist, words)

	println("Words in the book that are not in the word list:")
	for word in keys(diff)
		print(word, " ")
	end
```

Here are some of the results from Emma. Words in the book that are not in the word list: outree quicksighted outwardly adelaide rencontre jeffereys unreserved dixons betweens ... Some of these words are names and possessives. Others, like “rencontre”, are no longer in common use. But a few are common words that should really be in the list!

### Exercise 13-6

Julia provides a data structure called **Set** that provides many common set operations. You can read about them in `Collections and Data Structures`, or read the documentation at https://docs.julialang.org/en/v1/base/collections/#Set-Like-Collections-1. Write a program that uses set subtraction to find words in the book that are not in the word list.

In [29]:
function subtract(book::Array{String,1}, list::Array{String,1})
	return collect(setdiff!(Set(book), list))
end

subtract (generic function with 2 methods)

In [30]:
subtract(emma_words, collect(keys(wordlist)))

922-element Vector{String}:
 "bateses"
 "liv"
 "ungraciously"
 "singleminded"
 "bella"
 "ladys"
 "bowwindow"
 "churchills"
 "\\"
 "b"
 ⋮
 "efmma"
 "poultryhouse"
 "epenses"
 "williams"
 "unepected"
 "friday"
 "anne"
 "moderatesized"
 "twentyone"

### Two things to note

First, we already used the name **subtract** to define a function given by the signature:

```Julia
	function subtract(book::Dict, list::Dict)
```

Here we defined a function also called **subtracte** but with a different signature:

```Julia
	function subtract(book::Array{String,1}, list::Array{String, 1})
```

This is out first practical encounter with so-called **multiple dispatch** in Julia. Julia distinguishes between **methods** associated with a given function name and chooses the the appropriate method based on the **input types** in the function call, when appropriately defined in the **function signatures**. Julia announces this by echoing **dictionary_difference (generic function with 2 methods)**.

Second, text processing is messy because of a natural lack of standardization and often requires that we operate with Julia's more precise tools, typically all the way down to the character level by using various specifically coded processing tools relevant to the circumstances. Julia's way of dealing with **regular expressions (regex)** is a good place to continue learning.

### Random words

To choose a random word from the histogram, the simplest algorithm is to build an array with multiple copies of each word, according to the observed frequency, and then choose from the array:

```Julia
	function randomword(h)
		t = []
		for (word, freq) in h
			for i in 1:freq
				push!(t, word)
			end
		end
		rand(t)
	end
```

This algorithm works, but it is not very efficient; each time you choose a random word, it rebuilds the array, which is as big as the original book. An obvious improvement is to build the array once and then make multiple selections, but the array is still big. An alternative is:

(1) Use keys to get an array of the words in the book.

(2) Build an array that contains the cumulative sum of the word frequencies (see Exercise 10-2). The last item in this array should be the total number of words in the book, n.

(3) Choose a random number from 1 to n and use a bisection search (see Exercise 10-10) to find the index where the random number would be inserted in the cumulative sum.

(4) Use the index to find the corresponding word in the word array.

### Exercise 13-7

Write a program that uses this algorithm to choose a random word from the book.

Here is slightly different approach:

In [31]:
function clr(wordlist:: Array{String, 1})
    cwordlist = Array{String, 1}()

    for k = 1:length(wordlist)

        if !( wordlist[k] == "" || wordlist[k] == "\\")

            push!(cwordlist, wordlist[k])
        end
    end
    return cwordlist
end


function wordcounts(book::Array{String, 1})
    counts = Dict{String,Int}()
    for word in book
        if word == "" || word == "\\"
            # do nothing
        else
            counts[word] = get!(counts, word, 0) + 1
        end
    end
    return counts
end


function csum_dict(hist::Dict{String, Int})
    cshist = Dict()
    cs = 0
    for key in keys(hist)        # ks and vs are listed in corresponding order
        cs += hist[key]
        cshist[cs] = key
    end
    return cshist
end


function rnd_wrd(c_hst::Dict)

    M = maximum(keys(c_hst))
    rnd = rand(1:M)

    sc_hst = sort_on_ks(c_hst, false) 		# see code just prior to Exercise 13-3

    ind = findfirst(k -> r < k[1], sc_hst)

    return sc_hst[ind][2]
end

rnd_wrd (generic function with 1 method)

### Markov analysis

If you choose words from the book at random, you can get a sense of the vocabulary, but you probably will not get a sentence:

	this the small regard harriet which knightley's it most things

A series of random words seldom makes sense because there is no relationship between successive words. For example, in a real sentence you would expect an article like “the” to be followed by an adjective or a noun, and probably not a verb or adverb.

One way to measure these kinds of relationships is Markov analysis, which characterizes, for a given sequence of words, the probability of the words that might come next. For example, the song Eric, the Half a Bee (by Monty Python) begins:

	Half a bee, philosophically,
	Must, ipso facto, half not be.
	But half the bee has got to be
	Vis a vis, its entity. D’you see?

	But can a bee be said to be
	Or not to be an entire bee
	When half the bee is not a bee
	Due to some ancient injury?

In this text, the phrase “half the” is always followed by the word “bee”, but the phrase “the bee” might be followed by either “has” or “is”.

The result of Markov analysis is a mapping from each prefix (like “half the” and “the bee”) to all possible suffixes (like “has” and “is”). Given this mapping, you can generate a random text by starting with any prefix and choosing at random from the possible suffixes. Next, you can combine the end of the prefix and the new suffix to form the next prefix, and repeat.

For example, if you start with the prefix “Half a”, then the next word has to be “bee”, because the prefix only appears once in the text. The next prefix is “a bee”, so the next suffix might be “philosophically”, “be” or “due”. In this example the length of the prefix is always two, but you can do Markov analysis with any prefix length.

### Exercise 13-8

Write a program to read a text from a file and perform Markov analysis. The result should be a dictionary that maps from prefixes to a collection of possible suffixes. The collection might be an array, tuple, or dictionary; it is up to you to make an appropriate choice. You can test your program with prefix length two, but you should write the program in a way that makes it easy to try other lengths.

Add a function to the previous program to generate random text based on the Markov analysis. Here is an example from Emma with prefix length 2:

“He was very clever, be it sweetness or be angry, ashamed or only amused, at such a stroke. She had never thought of Hannah till you were never meant for me?" "I cannot make speeches, Emma:" he soon cut it all himself.”

The result is almost syntactically correct, but not quite. Semantically, it almost makes sense, but not quite. What happens if you increase the prefix length? Does the random text make more sense?

Once your program is working, you might want to try a mash-up: if you combine text from two or more books, the random text you generate will blend the vocabulary and phrases from the sources in interesting ways.

Credit: This case study is based on an example from Kernighan and Pike, The Practice of Programming, Addison-Wesley, 1999.

You should attempt this exercise before you go on.

### Data structures

In your solution to the previous exercises, you had to choose:

(1) How to represent the prefixes.

(2) How to represent the collection of possible suffixes.

(3) How to represent the mapping from each prefix to the collection of possible suffixes.

The last one is easy: a dictionary is the obvious choice for a mapping from keys to corresponding values. For the prefixes, the most obvious options are string, array of strings, or tuple of strings. For the suffixes, one option is an array; another is a histogram (dictionary).

How should you choose? 

The first step is to think about the operations you will need to implement for each data structure. For the prefixes, we need to be able to remove words from the beginning and add to the end. For example, if the current prefix is “Half a”, and the next word is “bee”, you need to be able to form the next prefix, “a bee”. Your first choice might be an array, since it is easy to add and remove elements.

For the collection of suffixes, the operations we need to perform include adding a new suffix (or increasing the frequency of an existing one), and choosing a random suffix. 
Adding a new suffix is equally easy for the array implementation or the histogram. Choosing a random element from an array is easy; choosing from a histogram is harder to do efficiently (see Exercise 13-7).

So far we have been talking mostly about ease of implementation, but there are other factors to consider in choosing data structures. One is run time. Sometimes there is a theoretical reason to expect one data structure to be faster than other; for example, the in operator is faster for dictionaries than for arrays, at least when the number of elements is large. But often you do not know ahead of time which implementation will be faster. One option is to implement both of them and see which is better. This approach is called **benchmarking**. 

A practical alternative is to choose the data structure that is easiest to implement, and then see if it is fast enough for the intended application. If so, there is no need to go on. If not, there are tools, like the **Profile module**, that can identify the places in a program that take the most time.

The other factor to consider is storage space. For example, using a histogram for the collection of suffixes might take less space because you only have to store each word once, no matter how many times it appears in the text. In some cases, saving space can also make your program run faster, and in the extreme, your program might not run at all if you run out of memory. But for many applications, space is a secondary consideration after run time.

The Julia package `DataStructures` (see https://github.com/JuliaCollections/DataStructures.jl) implements a variety of data structures.

### Debugging

When you are debugging a program, and especially if you are working on a hard bug, there are five things to try:

### Reading

Examine your code, read it back to yourself, and check that it says what you meant to say.

### Running

Experiment by making changes and running different versions. Often if you display the right thing at the right place in the program, the problem becomes obvious, but sometimes you have to build scaffolding.

### Ruminating

Take some time to think! What kind of error is it: syntax, runtime, or semantic? 

What information can you get from the error messages, or from the output of the program? What kind of error could cause the problem you are seeing? What did you change last, before the problem appeared?

### Rubberducking

If you explain the problem to someone else, you sometimes find the answer before you finish asking the question. Often you do not need the other person; you could just talk to a rubber duck. And that is the origin of the well-known strategy called rubber duck debugging https://en.wikipedia.org/wiki/Rubber_duck_debugging.

### Retreating

At some point, the best thing to do is back off, undoing recent changes, until you get back to a program that works and that you understand. Then you can start rebuilding.

Beginning programmers sometimes get stuck on one of these activities and forget the others. Each activity comes with its own failure mode. For example, reading your code might help if the problem is a typographical error, but not if the problem is a conceptual misunderstanding. If you do not understand what your program does, you can read it 100 times and never see the error, because the error is in your head.

Running experiments can help, especially if you run small, simple tests. But if you run experiments without thinking or reading your code, you might fall into a pattern I call “random walk programming”, which is the process of making random changes until the program does the right thing. Needless to say, random walk programming can take a long time.

You have to take time to think. Debugging is like an experimental science. You should have at least one hypothesis about what the problem is. If there are two or more possibilities, try to think of a test that would eliminate one of them.

But even the best debugging techniques will fail if there are too many errors, or if the code you are trying to fix is too big and complicated. Sometimes the best option is to retreat, simplifying the program until you get to something that works and that you understand. Beginning programmers are often reluctant to retreat because they cannot stand to delete a line of code. If it makes you feel better, copy your program into another file before you start stripping it down. Then you can copy the pieces back one at a time.

Finding a hard bug requires reading, running, ruminating, and sometimes retreating. If you get stuck on one of these activities, try the others.